In [6]:
import numpy as np
import pandas as pd

def fun1(x):#被积函数1，传入的x支持数组
    return 2.0 / 3.0 * (x ** 3) * np.exp(x ** 2)
def fun2(x):#被积函数2，传入的x支持数组
    return 2.0 * x / (x ** 2 - 3)

#梯形求积公式,a,b为积分上下限，n为等分数,fun为被积函数
def tixing(a, b, n, fun):
    h = (b - a)/n
    x = np.zeros([1,n+1])#记录各分点，x[0][0]为a,x[0][n]为b
    for i in range(n+1):
        x[0][i] = a + i * h
    y = fun(x)#计算各分点的函数值
    if n == 1:
        Tn = h / 2 * (y[0][0] + y[0][n])
    else:
        Tn = h / 2 * (y[0][0] + y[0][n] + 2 * np.sum(y[:,1:n]))#ndarray取和行列范围为[a,b)左开右闭形式
    return Tn

#辛普森求积公式,a,b为积分上下限，n为等分数,fun为被积函数
def xinpusen(a, b, n, fun):
    h = (b - a)/n
    x = np.zeros([1,n+1])#记录各分点，x[0][0]为a,x[0][n]为b
    x1_2 = np.zeros([1,n])#每个小区间的二分点
    for i in range(n+1):
        x[0][i] = a + i * h
    for i in range(n):
        x1_2[0][i] = (x[0][i] + x[0][i+1]) / 2.0
    y = fun(x)#计算各分点的函数值
    y1_2 = fun(x1_2)#计算每个小区间二分点的函数值
    if n == 1:
        Sn = h / 6 * (y[0][0] + y[0][n] + 2 * (y[0][0] + y[0][n]))
    else:
        Sn = h / 6 * (y[0][0] + y[0][n] + 2 * np.sum(y[:,1:n]) + 4 * np.sum(y1_2))#ndarray取和行列范围为[a,b)左开右闭形式
    return Sn

#龙贝格积分
#T表初始化，填入T[0][0]
def T_initial(w, a, b, fun):#w为维度,a、b为积分上下限，fun为函数解析式
    T = np.zeros([w,w])
    T[0][0] = (b-a) / 2 * (fun(a) + fun(b))
    return T

#一次加速
def one_N2O(T, x, i, h, fun):#T为T表，x为分点数组，i为累计加速次数,h为间隔长度
    len_x = x.shape[1]#未加速前的分点个数
    x1_2 = np.zeros([1,len_x - 1])#每个小区间的二分点
    for m in range(len_x - 1):
        x1_2[0][m] = (x[0][m] + x[0][m+1]) / 2.0
    T[i][0] = 1 / 2 * T[i-1][0] + h / 2 * np.sum(fun(x1_2))#T表开新的一行，并计算首列值
    for j in range (1 , i + 1):
        T[i][j] =  (4 ** j) / (4 ** j - 1) * T[i][j-1] - T[i-1][j-1] / (4 ** j - 1)
    total_point = x.shape[1] + x1_2.shape[1]#现在总分点的个数
    temp = []
    for i in range(total_point):
        if i % 2 == 0:#为偶数时
            temp.append(x[0][int(i / 2)])
        else:
            temp.append(x1_2[0][int((i - 1) / 2)])
    x = np.array(temp).reshape([1,total_point])
    return T, x,  h / 2.0

In [3]:
#复合梯形公式
a = 1
b = 2
cankao = np.exp(4)#参考值

initial = tixing(a, b, 1, fun1)
ebs = initial - 0
i = 1
while (ebs >= 0.000000005):
    i = i+1
    ebs = np.abs(tixing(a, b, i, fun1) - tixing(a, b, i+1, fun1))
print("复合梯形公式等分数：", i+1)
print("复合梯形公式所得积分值：",tixing(1, 2, i+1, fun1))
print("复合梯形公式迭代的绝对误差：",np.abs(tixing(a, b, i, fun1) - tixing(a, b, i+1, fun1)))
print("复合梯形公式与参考值的绝对误差：",np.abs(cankao-tixing(a, b, i+1, fun1)))



等分数： 3759
所得复合梯形积分值： 54.59815942495601
迭代的绝对误差： 4.998938152311894e-09
与参考值的绝对误差： 9.391811772729852e-06


In [4]:
#复合辛普森公式
a = 1
b = 2
cankao = np.exp(4)#参考值

initial = xinpusen(a, b, 2, fun1)
ebs = initial - 0
i = 1
while (ebs >= 0.000000005):
    i = i+1
    ebs = np.abs(xinpusen(a, b, i, fun1) - xinpusen(a, b, i+1, fun1))
print("复合辛普森公式等分数：",i+1)
print("复合辛普森公式所得积分值：",xinpusen(1, 2, i+1, fun1))
print("复合辛普森公式迭代的绝对误差：",np.abs(xinpusen(a, b, i, fun1) - xinpusen(a, b, i+1, fun1)))
print("复合辛普森公式与参考值的绝对误差：",np.abs(cankao - xinpusen(a, b, i+1, fun1)))

复合辛普森公式等分数： 110
复合辛普森公式所得积分值： 54.5981501622893
复合辛普森公式迭代的绝对误差： 4.8046544520730095e-09
复合辛普森公式与参考值的绝对误差： 1.291450644202996e-07


In [8]:
#龙贝格积分
w = 10#T表维度
a = 1
b = 2
cankao = np.exp(4)#参考值
eps = 0.000000005
x = np.array([a,b]).reshape([1,2])#初始化分点
h = b - a

T = T_initial(w, a, b, fun1)#初始化T表
T, x, h = one_N2O(T, x, 1, h, fun1)#先加一次速
n = 1#累计加速次数
while (np.abs(T[n][n] - T[n - 1][n - 1]) >= eps):
    n = n + 1
    T, x, h = one_N2O(T, x, n, h, fun1)#加一次速
print("加速次数：",n)
print("龙贝格算法所得积分值：",T[n][n])
print("T表：")
print(T)
print("龙贝格算法迭代的绝对误差：",np.abs(T[n][n] - T[n - 1][n - 1]))
print("龙贝格算法与参考值的绝对误差：",np.abs(T[n][n] - cankao))

加速次数： 7
龙贝格算法所得积分值： 54.59815003314422
T表：
[[146.5011607    0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [ 83.92428316  63.06532399   0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [ 62.61319631  55.50950069  55.00577913   0.           0.
    0.           0.           0.           0.           0.        ]
 [ 56.65349937  54.66693373  54.61076259  54.60449249   0.
    0.           0.           0.           0.           0.        ]
 [ 55.11538708  54.60268299  54.5983996   54.59820337  54.5981787
    0.           0.           0.           0.           0.        ]
 [ 54.72767471  54.59843725  54.5981542   54.59815031  54.5981501
   54.59815007   0.           0.           0.           0.        ]
 [ 54.63054471  54.59816805  54.5981501   54.59815003  54.59815003
   54.59815003  54.59815003   0.           0.           0.        ]
 [ 54.60624955  54.59815116  54.59815003  54.59

In [13]:
#复合梯形公式
a = 2
b = 3
cankao = np.log(6)

initial = tixing(a, b, 1, fun2)
ebs = initial - 0
i = 1
while (ebs >= 0.000000005):
    i = i+1
    ebs = np.abs(tixing(a, b, i, fun2) - tixing(a, b, i+1, fun2))
print("复合梯形公式等分数：", i+1)
print("复合梯形公式所得积分值：",tixing(a, b, i+1, fun2))
print("复合梯形公式迭代的绝对误差：",np.abs(tixing(a, b, i, fun2) - tixing(a, b, i+1, fun2)))
print("复合梯形公式与参考值的绝对误差：",np.abs(cankao-tixing(a, b, i+1, fun2)))


复合梯形公式等分数： 764
复合梯形公式所得积分值： 1.7917613728017425
复合梯形公式迭代的绝对误差： 4.992964930394805e-09
复合梯形公式与参考值的绝对误差： 1.9035736875672171e-06


In [15]:
#复合辛普森公式
a = 2
b = 3
cankao = np.log(6)

initial = xinpusen(a, b, 2, fun2)
ebs = initial - 0
i = 1
while (ebs >= 0.000000005):
    i = i+1
    ebs = np.abs(xinpusen(a, b, i, fun2) - xinpusen(a, b, i+1, fun2))
print("复合梯形公式等分数：", i+1)
print("复合梯形公式所得积分值：",xinpusen(a, b, i+1, fun2))
print("复合梯形公式迭代的绝对误差：",np.abs(xinpusen(a, b, i, fun2) - xinpusen(a, b, i+1, fun2)))
print("复合梯形公式与参考值的绝对误差：",np.abs(cankao-xinpusen(a, b, i+1, fun2)))

复合梯形公式等分数： 51
复合梯形公式所得积分值： 1.7917595286610115
复合梯形公式迭代的绝对误差： 4.8909591932044805e-09
复合梯形公式与参考值的绝对误差： 5.943295655619352e-08


In [16]:
#龙贝格积分
w = 10#T表维度
a = 2
b = 3
cankao = np.log(6)#参考值
eps = 0.000000005
x = np.array([a,b]).reshape([1,2])#初始化分点
h = b - a

T = T_initial(w, a, b, fun2)#初始化T表
T, x, h = one_N2O(T, x, 1, h, fun2)#先加一次速
n = 1#累计加速次数
while (np.abs(T[n][n] - T[n - 1][n - 1]) >= eps):
    n = n + 1
    T, x, h = one_N2O(T, x, n, h, fun2)#加一次速
print("加速次数：",n)
print("龙贝格算法所得积分值：",T[n][n])
print("T表：")
print(T)
print("龙贝格算法迭代的绝对误差：",np.abs(T[n][n] - T[n - 1][n - 1]))
print("龙贝格算法与参考值的绝对误差：",np.abs(T[n][n] - cankao))

加速次数： 7
龙贝格算法所得积分值： 1.7917594692290668
T表：
[[2.5        0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [2.01923077 1.85897436 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [1.85643979 1.80217613 1.79838959 0.         0.         0.
  0.         0.         0.         0.        ]
 [1.80876065 1.7928676  1.79224703 1.79214953 0.         0.
  0.         0.         0.         0.        ]
 [1.79607573 1.79184743 1.79177942 1.791772   1.79177052 0.
  0.         0.         0.         0.        ]
 [1.79284301 1.79176544 1.79175997 1.79175966 1.79175961 1.7917596
  0.         0.         0.         0.        ]
 [1.79203064 1.79175985 1.79175948 1.79175947 1.79175947 1.79175947
  1.79175947 0.         0.         0.        ]
 [1.79182728 1.79175949 1.79175947 1.79175947 1.79175947 1.79175947
  1.79175947 1.79175947 0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.   